In [1]:
print("hello world")

hello world


In [2]:
import torch
isAvailable = torch.cuda.is_available()
print("Is cuda available ",isAvailable)

Is cuda available  False


In [1]:
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
config = {
        "batch_size": tune.choice([8, 16, 32]),
        "learning_rate": tune.loguniform(1e-4, 1e-1),
        # red convolucional
        "cant_filtros_conv1": tune.choice([6, 12, 18]),
        "kernel_size_maxpool1": tune.choice([2, 3]),
        "cant_filtros_conv2": tune.choice([16, 22, 28]),
        "kernel_size_maxpool2": tune.choice([2, 3]),
        "full_l1": tune.choice([120, 140, 160]),

        "full_l2": tune.choice([84, 104, 124])
    }

In [2]:
"""
probar que absolutamente todas las posibles combinaciones no tiran fallo.
"""
import torch
from modelo_convolucional import instanciarRed
passed = 0
not_passed =0
fail_configs = []
for conv1 in [6, 12, 18]:
    for maxpool1 in [2, 3]:
        for conv2 in [16, 22, 28]:
            for maxpool2 in [2, 3]:
                for l1 in [120, 140, 160]:
                    for l2 in [84, 104, 124]:
                        config = {
                            "cant_filtros_conv1": conv1,
                            "kernel_size_maxpool1": maxpool1,
                            "cant_filtros_conv2": conv2,
                            "kernel_size_maxpool2": maxpool2,
                            "full_l1": l1,
                            "full_l2": l2,
                        }

                        model = instanciarRed(config)
                        random = torch.rand(1, 3, 32, 32)
                        try:
                            model(random)
                            passed+=1
                        except Exception as e:
                            not_passed+=1
                            print(config)
                            fail_configs.append(config)
                            #raise e
print("passed ",passed)
print("not passed ",not_passed)
print(not_passed/(passed+not_passed)*100,"% not passed")

{'cant_filtros_conv1': 6, 'kernel_size_maxpool1': 2, 'cant_filtros_conv2': 16, 'kernel_size_maxpool2': 2, 'full_l1': 120, 'full_l2': 84}
{'cant_filtros_conv1': 6, 'kernel_size_maxpool1': 2, 'cant_filtros_conv2': 16, 'kernel_size_maxpool2': 2, 'full_l1': 120, 'full_l2': 104}
{'cant_filtros_conv1': 6, 'kernel_size_maxpool1': 2, 'cant_filtros_conv2': 16, 'kernel_size_maxpool2': 2, 'full_l1': 120, 'full_l2': 124}
{'cant_filtros_conv1': 6, 'kernel_size_maxpool1': 2, 'cant_filtros_conv2': 16, 'kernel_size_maxpool2': 2, 'full_l1': 140, 'full_l2': 84}
{'cant_filtros_conv1': 6, 'kernel_size_maxpool1': 2, 'cant_filtros_conv2': 16, 'kernel_size_maxpool2': 2, 'full_l1': 140, 'full_l2': 104}
{'cant_filtros_conv1': 6, 'kernel_size_maxpool1': 2, 'cant_filtros_conv2': 16, 'kernel_size_maxpool2': 2, 'full_l1': 140, 'full_l2': 124}
{'cant_filtros_conv1': 6, 'kernel_size_maxpool1': 2, 'cant_filtros_conv2': 16, 'kernel_size_maxpool2': 2, 'full_l1': 160, 'full_l2': 84}
{'cant_filtros_conv1': 6, 'kernel_siz

In [2]:
import torch
from modelo_convolucional import instanciarRed
# config = fail_configs[0]
# print(config)
config = {
    "cant_filtros_conv1": 6,
    "kernel_size_maxpool1": 2,
    "cant_filtros_conv2": 16,
    "kernel_size_maxpool2": 2,
    "full_l1": 120,
    "full_l2": 84,
}

model = instanciarRed(config)

random = torch.rand(1, 3, 32, 32)
model(random)



RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x400 and 576x120)

In [4]:
from icecream import ic
from torch import nn
ic(model.o_conv2d)

conv =nn.Conv2d(in_channels=3, out_channels=model.o_conv2d[0],
                      kernel_size=5, stride=1, padding=1)
ic(random.shape)
rta =conv(random)

ic(rta.shape)

ic| model.o_conv2d: [6, 30]
ic| random.shape: torch.Size([1, 3, 32, 32])
ic| rta.shape: torch.Size([1, 6, 30, 30])


torch.Size([1, 6, 30, 30])